In [1]:
from wikidataintegrator import wdi_core, wdi_login
from wikidataintegrator.ref_handlers import update_retrieved_if_new_multiple_refs
import pandas as pd
import numpy as np
import requests

In [84]:
kantonZH = pd.read_csv('kantonZH.csv')    
kantonZH.head()
#[(city['Name']=='Adlikon')]
cityZH = pd.read_csv('cityZH.csv')    
cityZH.head()

,Unnamed: 0,date,id,name,population,wikidata_id,check,bool
0,0,2018-12-31,123,Hirzenbach,12801,Q693374,2018-12-31---Q693374,False
1,1,2017-12-31,123,Hirzenbach,12627,Q693374,2017-12-31---Q693374,False
2,2,2016-12-31,123,Hirzenbach,12463,Q693374,2016-12-31---Q693374,False
3,3,2015-12-31,123,Hirzenbach,11930,Q693374,2015-12-31---Q693374,False
4,4,2014-12-31,123,Hirzenbach,11679,Q693374,2014-12-31---Q693374,False


In [75]:
print("Logging in...")
#if "WDUSER" in os.environ and "WDPASS" in os.environ:
#    WDUSER = os.environ['WDUSER']
#    WDPASS = os.environ['WDPASS']
#else:
#    raise ValueError("WDUSER and WDPASS must be specified in local.py or as environment variables")
login = wdi_login.WDLogin("Tlzh", "wikiwiki")

Logging in...
https://www.wikidata.org/w/api.php
Successfully logged in as Tlzh


In [86]:
# Quelle setzen
# Quartier 
#  Q4115189|P1082|805|P585|+2017-12-31T00:00:00Z/9|P459|Q12483|S854|"https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_seit1970_od3240"
from datetime import datetime
import copy
def create_reference():
    timeStringNow = datetime.now().strftime("+%Y-%m-%dT00:00:00Z")
    # Timestamp P585
    refRetrieved = wdi_core.WDTime(timeStringNow, prop_nr="P813", is_reference=True)
    # URL Source P854
    refURL = wdi_core.WDUrl(value="https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_seit1970_od3240", 
                            prop_nr="P854", is_reference=True)

    return [refRetrieved, refURL]

def create_qualifier(insertDate):
    # Point in time P585
    refRetrieved = wdi_core.WDTime(insertDate, prop_nr="P585", is_qualifier=True)
    # Statistics P459
    bestimmungsmethode = wdi_core.WDItemID(value='Q12483', prop_nr='P459', is_qualifier=True)

    return [refRetrieved, bestimmungsmethode]


In [87]:
reference = create_reference()
insertDate = datetime(2017,12,31).strftime("+%Y-%m-%dT00:00:00Z")
qualfier = create_qualifier(insertDate)
statement = [wdi_core.WDQuantity(value=23797, prop_nr="P1082", references=[copy.deepcopy(reference)], 
                                 rank='preferred', qualifiers=copy.deepcopy(qualfier), check_qualifier_equality=False)]
# Q von Höngg --> quartier
wd_item = wdi_core.WDItemEngine(data=statement,wd_item_id='Q455496', new_item=False, fast_run=False, append_value=['P1082'])
wd_item.write(login)

'Q455496'